# Lab Assignment 2 - Part B: k-Nearest Neighbor Classification
Please refer to the `README.pdf` for full laboratory instructions.


## Problem Statement
In this part, you will implement the k-Nearest Neighbor (k-NN) classifier and evaluate it on two datasets:
- **Lenses Dataset**: A small dataset for contact lens prescription
- **Credit Approval (CA) Dataset**: Credit card application data with binary labels (+/-)

### Your Tasks
1. **Preprocess the data**: Handle missing values and normalize features
2. **Implement k-NN** with L2 distance
3. **Evaluate** on both datasets for different values of k
4. **Discuss** your results

### Datasets
The data files are located in the `credit 2017/` folder:
- `lenses.training`, `lenses.testing`
- `crx.data.training`, `crx.data.testing`
- `crx.names` (describes the features)


## Setup


In [1]:
# Library declarations
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter


In [ ]:
# Data paths
DATA_PATH = "credit 2017/"

# Load Lenses data
def load_lenses_data():
    """Load the lenses dataset."""
    train_data = np.loadtxt(DATA_PATH + "lenses.training", delimiter=',')
    test_data = np.loadtxt(DATA_PATH + "lenses.testing", delimiter=',')
    
    # First column is ID, last column is label
    X_train = train_data[:, 1:-1]
    y_train = train_data[:, -1]
    X_test = test_data[:, 1:-1]
    y_test = test_data[:, -1]
    
    return X_train, y_train, X_test, y_test

# Load Credit Approval data
def load_credit_data():
    """
    Load the Credit Approval dataset.
    Note: This dataset contains missing values (?) and mixed types.
    You will need to preprocess it.
    """
    # TODO: Implement data loading
    # The data is comma-separated
    # Missing values are marked with '?'
    # Last column is the label ('+' or '-')
    train_file = DATA_PATH + "crx.data.training"
    test_file = DATA_PATH + "crx.data.testing"

    train_raw = np.genfromtxt(train_file, delimiter=',', dtype=str)
    test_raw = np.genfromtxt(test_file, delimiter=',', dtype=str)

    X_train = train_raw[:, :-1]
    y_train = train_raw[:, -1]
    X_test = test_raw[:, :-1]
    y_test = test_raw[:, -1]

    return X_train, y_train, X_test, y_test

# Test loading lenses data
X_train_lenses, y_train_lenses, X_test_lenses, y_test_lenses = load_lenses_data()
print(f"Lenses - Train: {X_train_lenses.shape}, Test: {X_test_lenses.shape}")


Lenses - Train: (18, 3), Test: (6, 3)


## Task 1: Data Preprocessing
For the Credit Approval dataset, you need to:
1. **Handle missing values** (marked with '?'):
   - Categorical features: replace with mode/median
   - Numerical features: replace with label-conditioned mean
2. **Normalize features** using z-scaling:
   $$z_i^{(m)} = \frac{x_i^{(m)} - \mu_i}{\sigma_i}$$

Document exactly how you handle each feature!


In [11]:
def preprocess_credit_data(train_file, test_file):
    """
    Preprocess the Credit Approval dataset.
    
    Steps:
    1. Load the data
    2. Handle missing values
    3. Encode categorical variables
    4. Normalize numerical features
    
    Returns:
    --------
    X_train, y_train, X_test, y_test : numpy arrays
    """
    # TODO: Implement preprocessing
    # Hint: Read crx.names to understand the features
    # Feature types (from crx.names):
    # A1: categorical (b, a)
    # A2: continuous
    # A3: continuous
    # A4: categorical (u, y, l, t)
    # A5: categorical (g, p, gg)
    # A6: categorical (c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff)
    # A7: categorical (v, h, bb, j, n, z, dd, ff, o)
    # A8: continuous
    # A9: categorical (t, f)
    # A10: categorical (t, f)
    # A11: continuous
    # A12: categorical (t, f)
    # A13: categorical (g, p, s)
    # A14: continuous
    # A15: continuous
    # A16: label (+, -)
    
    # categorical features: mode
    train_raw = np.genfromtxt(train_file, delimiter=',', dtype=str)
    test_raw = np.genfromtxt(test_file, delimiter=',', dtype=str)

    X_train_raw = train_raw[:, :-1]
    y_train = train_raw[:, -1]
    X_test_raw = test_raw[:, :-1]
    y_test = test_raw[:, -1]

    numerical_idx = [1, 2, 7, 10, 13, 14]
    categorical_idx = [i for i in range(15) if i not in numerical_idx]

    # Missing value imputation
    # Categorical: replace '?' with mode (computed from training, per feature)
    for j in categorical_idx:
        col = X_train_raw[:, j]
        non_missing = col[col != '?']
        if len(non_missing) == 0:
            mode_val = '0'
        else:
            mode_val = Counter(non_missing).most_common(1)[0][0]
        X_train_raw[col == '?', j] = mode_val
        X_test_raw[X_test_raw[:, j] == '?', j] = mode_val

    # Numerical: replace '?' with label-conditioned mean from training
    label_set = np.unique(y_train)
    class_means = {lab: {} for lab in label_set}
    for j in numerical_idx:
        for lab in label_set:
            mask = (y_train == lab) & (X_train_raw[:, j] != '?')
            vals = X_train_raw[mask, j].astype(float)
            class_means[lab][j] = float(vals.mean()) if vals.size > 0 else 0.0

        missing_mask_train = (X_train_raw[:, j] == '?')
        for i in np.where(missing_mask_train)[0]:
            lab = y_train[i]
            X_train_raw[i, j] = str(class_means[lab][j])

        missing_mask_test = (X_test_raw[:, j] == '?')
        for i in np.where(missing_mask_test)[0]:
            lab = y_test[i]
            if lab in class_means:
                X_test_raw[i, j] = str(class_means[lab][j])
            else:
                vals_all = X_train_raw[X_train_raw[:, j] != '?', j].astype(float)
                X_test_raw[i, j] = str(float(vals_all.mean()) if vals_all.size > 0 else 0.0)

    # Encode categorical variables (map each category to an integer, per feature, using training)
    X_train_enc = X_train_raw.copy()
    X_test_enc = X_test_raw.copy()

    for j in categorical_idx:
        categories = np.unique(X_train_enc[:, j])
        mapping = {cat: idx for idx, cat in enumerate(categories)}
        X_train_enc[:, j] = np.vectorize(lambda v: mapping[v])(X_train_enc[:, j])
        def map_test(v):
            return mapping[v] if v in mapping else len(mapping)
        X_test_enc[:, j] = np.vectorize(map_test)(X_test_enc[:, j])

    X_train = X_train_enc.astype(float)
    X_test = X_test_enc.astype(float)
    X_train, X_test = z_normalize(X_train, X_test, numerical_idx)

    return X_train, y_train, X_test, y_test


def z_normalize(X_train, X_test, feature_indices):
    """
    Apply z-score normalization to specified features.
    
    Parameters:
    -----------
    X_train, X_test : numpy arrays
    feature_indices : list of indices for numerical features
    
    Returns:
    --------
    X_train_normalized, X_test_normalized : numpy arrays
    """
    # TODO: Implement z-normalization
    X_train_norm = X_train.copy().astype(float)
    X_test_norm = X_test.copy().astype(float)

    for j in feature_indices:
        mu = X_train_norm[:, j].mean()
        sigma = X_train_norm[:, j].std()
        if sigma == 0:
            sigma = 1.0
        X_train_norm[:, j] = (X_train_norm[:, j] - mu) / sigma
        X_test_norm[:, j] = (X_test_norm[:, j] - mu) / sigma

    return X_train_norm, X_test_norm


## Task 2: Implement k-NN Classifier
Implement k-NN with L2 (Euclidean) distance:
$$\mathcal{D}_{L2}(\mathbf{a}, \mathbf{b}) = \sqrt{\sum_i (a_i - b_i)^2}$$

For **categorical attributes**, use:
- Distance = 1 if values are different
- Distance = 0 if values are the same


In [13]:
def l2_distance(a, b):
    """
    Compute L2 (Euclidean) distance between two vectors.
    
    Parameters:
    -----------
    a, b : numpy arrays of same shape
    
    Returns:
    --------
    distance : float
    """
    # TODO: Implement L2 distance
    a = np.asarray(a)
    b = np.asarray(b)

    if a.shape[0] == 15 and b.shape[0] == 15:
        numerical_idx = [1, 2, 7, 10, 13, 14]
        categorical_idx = [i for i in range(15) if i not in numerical_idx]
        dist_sq = 0.0
        # numerical part
        for j in numerical_idx:
            dist_sq += (a[j] - b[j]) ** 2
        # categorical part
        for j in categorical_idx:
            dist_sq += 0.0 if a[j] == b[j] else 1.0
        return float(np.sqrt(dist_sq))
    return float(np.sqrt(np.sum((a - b) ** 2)))


def knn_predict(X_train, y_train, X_test, k):
    """
    Predict labels for test data using k-NN.
    
    Parameters:
    -----------
    X_train : numpy array of shape (n_train, n_features)
    y_train : numpy array of shape (n_train,)
    X_test : numpy array of shape (n_test, n_features)
    k : int, number of neighbors
    
    Returns:
    --------
    predictions : numpy array of shape (n_test,)
    """
    # TODO: Implement k-NN prediction
    # For each test sample:
    #   1. Compute distance to all training samples
    #   2. Find k nearest neighbors
    #   3. Predict using majority voting
    X_train = np.asarray(X_train)
    X_test = np.asarray(X_test)
    y_train = np.asarray(y_train)

    predictions = []
    for x in X_test:
        dists = []
        for i in range(X_train.shape[0]):
            d = l2_distance(X_train[i], x)
            dists.append((d, y_train[i]))

        dists.sort(key=lambda t: t[0])
        neighbors = dists[:k]
        votes = [lab for _, lab in neighbors]

        pred = Counter(votes).most_common(1)[0][0]
        predictions.append(pred)
    return np.array(predictions)


def compute_accuracy(y_true, y_pred):
    """
    Compute classification accuracy.
    
    Returns:
    --------
    accuracy : float (between 0 and 1)
    """
    # TODO: Implement accuracy computation
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    return float(np.mean(y_true == y_pred))


## Task 3: Evaluate on Lenses Dataset
Test your k-NN implementation on the Lenses dataset for different values of k.


In [20]:
# TODO: Evaluate k-NN on Lenses dataset
# Try different values of k (e.g., 1, 3, 5, 7)

k_values = [1, 3, 5, 7]
lenses_results = []

for k in k_values:
    predictions = knn_predict(X_train_lenses, y_train_lenses, X_test_lenses, k)
    accuracy = compute_accuracy(y_test_lenses, predictions)
    lenses_results.append((k, accuracy))
    print(f"k={k}: Accuracy = {accuracy:.4f}")


k=1: Accuracy = 1.0000
k=3: Accuracy = 1.0000
k=5: Accuracy = 1.0000
k=7: Accuracy = 1.0000


## Task 4: Evaluate on Credit Approval Dataset
First preprocess the data, then evaluate k-NN.


In [21]:
# TODO: Preprocess Credit Approval data
X_train_credit, y_train_credit, X_test_credit, y_test_credit = preprocess_credit_data(
    DATA_PATH + "crx.data.training",
    DATA_PATH + "crx.data.testing"
)


In [22]:
# TODO: Evaluate k-NN on Credit Approval dataset
k_values = [1, 3, 5, 7]
credit_results = []

for k in k_values:
    predictions = knn_predict(X_train_credit, y_train_credit, X_test_credit, k)
    accuracy = compute_accuracy(y_test_credit, predictions)
    credit_results.append((k, accuracy))
    print(f"k={k}: Accuracy = {accuracy:.4f}")


k=1: Accuracy = 0.8116
k=3: Accuracy = 0.8478
k=5: Accuracy = 0.8333
k=7: Accuracy = 0.8478


## Summary and Discussion

### Results Table

| Dataset | k=1 | k=3 | k=5 | k=7 |
|---------|-----|-----|-----|-----|
| Lenses | 1.0000 | 1.0000 | 1.0000 | 1.0000 |
| Credit Approval | 0.8116 | 0.8478 | 0.8333 | 0.8478 |

### Discussion
*Answer these questions:*
1. Which value of k works best for each dataset? Why do you think that is?

    From the results, k = 3 and k = 7 achieve the highest accuracy (0.8478) on the Credit Approval dataset. Among them, k = 3 is preferred because it achieves high accuracy while using fewer neighbors, making it less likely to oversmooth class boundaries.

2. How did preprocessing affect your results on the Credit Approval dataset?

    Increasing k does not guarantee better performance in kNN. As k becomes larger, the classifier considers more distant neighbors, which may belong to different classes. This can introduce incorrect votes and reduce accuracy.

3. What are the trade-offs of using different values of k?

    The Lenses dataset is very small and has well-separated classes. Because kNN is a memory-based classifier, it can easily memorize the training samples and correctly classify the test samples, resulting in perfect accuracy for all tested values of k.

4. What did you learn from this exercise?

    This exercise helped me better understand how k-Nearest Neighbor works in practice, especially the impact of the parameter k on model performance. By testing different values of k, I observed that accuracy does not necessarily improve monotonically as k increases. 
